# U-Net

### U-Net Overview
U-Net은 주로 의료 영상에서 사용되는 이미지 분할을 위한 Convolution Neural Network, 즉 CNNs입니다. U-Net의 주요 특징은 U자형의 대칭 구조로, 인코딩 경로와 디코딩 경로로 구성되어 있습니다. 인코딩 경로는 입력 이미지를 점점 더 작은 차원으로 축소하고, 디코딩 경로는 이를 다시 원래 크기로 확장하면서 정확한 예측을 만들어냅니다. 인코딩과 디코딩 경로 사이에는 skip connection이 있어, 인코딩 단계에서 추출된 특징을 디코딩 단계에서 다시 사용하여 더 좋은 결과를 얻을 수 있습니다.

### U-Net Architecture

- 인코딩 경로 (Contracting Path):
    - 컨볼루션 레이어와 풀링 레이어를 사용하여 점진적으로 이미지의 차원을 줄입니다.
- 디코딩 경로 (Expansive Path):
    - 업샘플링과 컨볼루션 레이어를 사용하여 이미지의 차원을 원래 크기로 복원합니다.
- Skip Connections:
    - 인코딩 경로의 각 단계에서 얻은 특징 맵을 디코딩 경로의 대응하는 단계에 결합합니다.

### PyTorch Implementation

```python
import torch
import torch.nn as nn
import torch.nn.functional as F

class UNet(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(UNet, self).__init__()
        
        # 인코딩 경로 (Contracting Path)
        self.encoder1 = self.contracting_block(in_channels, 64)   # 첫 번째 인코딩 블록: in_channels -> 64 채널
        self.encoder2 = self.contracting_block(64, 128)           # 두 번째 인코딩 블록: 64 -> 128 채널
        self.encoder3 = self.contracting_block(128, 256)          # 세 번째 인코딩 블록: 128 -> 256 채널
        self.encoder4 = self.contracting_block(256, 512)          # 네 번째 인코딩 블록: 256 -> 512 채널
        
        # 중심부 (Bottom of the U)
        self.bottom = self.contracting_block(512, 1024)           # 중심부 블록: 512 -> 1024 채널
        
        # 디코딩 경로 (Expansive Path)
        self.upconv4 = self.expansive_block(1024, 512)            # 첫 번째 디코딩 블록: 1024 -> 512 채널
        self.upconv3 = self.expansive_block(512, 256)             # 두 번째 디코딩 블록: 512 -> 256 채널
        self.upconv2 = self.expansive_block(256, 128)             # 세 번째 디코딩 블록: 256 -> 128 채널
        self.upconv1 = self.expansive_block(128, 64)              # 네 번째 디코딩 블록: 128 -> 64 채널
        
        # 마지막 출력 레이어
        self.final_conv = nn.Conv2d(64, out_channels, kernel_size=1) # 최종 출력 레이어: 1x1 컨볼루션, 64 -> out_channels
    
    def contracting_block(self, in_channels, out_channels):
        """
        인코딩 블록 정의: Conv2d -> ReLU -> Conv2d -> ReLU
        패딩 없이 각 컨볼루션을 적용하여 출력 크기 축소
        """
        block = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3),
            nn.ReLU(inplace=True)
        )
        return block
    
    def expansive_block(self, in_channels, out_channels):
        """
        디코딩 블록 정의: ConvTranspose2d -> ReLU -> Conv2d -> ReLU -> Conv2d -> ReLU
        ConvTranspose2d를 통해 업샘플링 및 출력 크기 증가
        """
        block = nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size=2, stride=2),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, kernel_size=3),
            nn.ReLU(inplace=True)
        )
        return block
    
    def crop_and_concat(self, upsampled, bypass):
        """
        크기를 맞추기 위해 센터 크롭을 적용하고, 업샘플된 텐서와 인코딩 블록 출력을 결합
        """
        _, _, H, W = upsampled.size()
        bypass = F.center_crop(bypass, [H, W])
        return torch.cat((upsampled, bypass), dim=1)
    
    def forward(self, x):
        # 인코딩 경로 (Contracting Path)
        enc1 = self.encoder1(x)                                      # 첫 번째 인코딩 블록을 통해 특징 추출
        enc2 = self.encoder2(F.max_pool2d(enc1, kernel_size=2))      # 두 번째 인코딩 블록: max pooling 후 특징 추출
        enc3 = self.encoder3(F.max_pool2d(enc2, kernel_size=2))      # 세 번째 인코딩 블록: max pooling 후 특징 추출
        enc4 = self.encoder4(F.max_pool2d(enc3, kernel_size=2))      # 네 번째 인코딩 블록: max pooling 후 특징 추출
        
        # 중심부 (Bottom of the U)
        bottleneck = self.bottom(F.max_pool2d(enc4, kernel_size=2))  # 중심부 블록: max pooling 후 특징 추출
        
        # 디코딩 경로 (Expansive Path)
        dec4 = self.crop_and_concat(self.upconv4(bottleneck), enc4)  # 첫 번째 디코딩 블록: 업샘플링 후 결합
        dec3 = self.crop_and_concat(self.upconv3(dec4), enc3)        # 두 번째 디코딩 블록: 업샘플링 후 결합
        dec2 = self.crop_and_concat(self.upconv2(dec3), enc2)        # 세 번째 디코딩 블록: 업샘플링 후 결합
        dec1 = self.crop_and_concat(self.upconv1(dec2), enc1)        # 네 번째 디코딩 블록: 업샘플링 후 결합
        
        # 최종 출력 레이어
        return self.final_conv(dec1)                                 # 최종 출력 레이어를 통해 결과 반환


model = UNet(in_channels=3, out_channels=2)
```